# Install

## Install BenchMARL

In [ ]:
#@title
!git clone https://github.com/facebookresearch/BenchMARL

In [ ]:
#@title
%cd /content/BenchMARL

In [ ]:
#@title
!pip install -U torch torchvision
!pip install -e .

## Install Melting Pot

In [ ]:
#@title
!pip install dm-meltingpot

In [ ]:
#@title
!apt-get update
!apt-get install -y x11-utils python3-opengl xvfb
!pip install pyvirtualdisplay
import pyvirtualdisplay
display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
display.start()

# Launch from command line

To launch an experiment from the command line you can do

In [ ]:
!python benchmarl/run.py -m algorithm=mappo task=meltingpot/collaborative_cooking__circuit model=layers/cnn critic_model=\${model} seed=0,1 experiment.max_n_frames=30000 experiment.train_device=cuda experiment.sampling_device=cuda experiment.render=false experiment.buffer_device=cuda experiment.on_policy_n_envs_per_worker=1 "experiment.loggers=[]"


In [ ]:
from benchmarl.algorithms import MappoConfig
from benchmarl.environments import MeltingPotTask
from benchmarl.experiment import Experiment, ExperimentConfig
from benchmarl.models.cnn import CnnConfig
import torch.nn as nn

experiment_config = ExperimentConfig.get_from_yaml()
task = MeltingPotTask.COLLABORATIVE_COOKING__CIRCUIT.get_from_yaml()
algorithm_config = MappoConfig.get_from_yaml()
model_config = CnnConfig(
    cnn_num_cells=[32, 32],
    cnn_kernel_sizes=[3, 3],
    cnn_strides=[1, 1],
    cnn_paddings=[1, 1],
    cnn_activation_class=nn.ReLU,
    mlp_num_cells=[256, 256],
    mlp_layer_class=nn.Linear,
    mlp_activation_class=nn.Tanh,
)
critic_model_config = model_config

experiment_config.max_n_frames = 30000
experiment_config.evaluation_interval = 6000
experiment_config.on_policy_collected_frames_per_batch = 3000
experiment_config.loggers = []

experiment = Experiment(
    task=task,
    algorithm_config=algorithm_config,
    model_config=model_config,
    critic_model_config=critic_model_config,
    seed=0,
    config=experiment_config,
)
experiment.run()


You can run benchmarks as multi-runs like

In [ ]:
!python benchmarl/run.py -m algorithm=mappo,qmix,masac task=vmas/balance,vmas/sampling seed=0,1 experiment.max_n_frames=12000 "experiment.loggers=[]"

# Launch from a python script

You can also load and launch your experiments from within a script

In [ ]:
from benchmarl.algorithms import MappoConfig
from benchmarl.environments import VmasTask
from benchmarl.experiment import Experiment, ExperimentConfig
from benchmarl.models.mlp import MlpConfig

# Loads from "benchmarl/conf/experiment/base_experiment.yaml"
experiment_config = ExperimentConfig.get_from_yaml()
# Loads from "benchmarl/conf/task/vmas/balance.yaml"
task = VmasTask.BALANCE.get_from_yaml()
# Loads from "benchmarl/conf/algorithm/mappo.yaml"
algorithm_config = MappoConfig.get_from_yaml()
# Loads from "benchmarl/conf/model/layers/mlp.yaml"
model_config = MlpConfig.get_from_yaml()
critic_model_config = MlpConfig.get_from_yaml()

experiment_config.max_n_frames = 12_000
experiment_config.loggers = []

experiment = Experiment(
    task=task,
    algorithm_config=algorithm_config,
    model_config=model_config,
    critic_model_config=critic_model_config,
    seed=0,
    config=experiment_config,
)
experiment.run()

You can also run multiple experiments in a Benchmark.

In [ ]:
from benchmarl.algorithms import MappoConfig, MasacConfig, QmixConfig
from benchmarl.benchmark import Benchmark
from benchmarl.environments import VmasTask
from benchmarl.experiment import ExperimentConfig
from benchmarl.models.mlp import MlpConfig

# Loads from "benchmarl/conf/experiment/base_experiment.yaml"
experiment_config = ExperimentConfig.get_from_yaml()
# Loads from "benchmarl/conf/task/vmas"
tasks = [VmasTask.BALANCE.get_from_yaml(), VmasTask.SAMPLING.get_from_yaml()]
# Loads from "benchmarl/conf/algorithm"
algorithm_configs = [
    MappoConfig.get_from_yaml(),
    QmixConfig.get_from_yaml(),
    MasacConfig.get_from_yaml(),
]
# Loads from "benchmarl/conf/model/layers"
model_config = MlpConfig.get_from_yaml()
critic_model_config = MlpConfig.get_from_yaml()

experiment_config.max_n_frames = 12_000
experiment_config.loggers = []

benchmark = Benchmark(
    algorithm_configs=algorithm_configs,
    tasks=tasks,
    seeds={0, 1},
    experiment_config=experiment_config,
    model_config=model_config,
    critic_model_config=critic_model_config,
)
benchmark.run_sequential()